<a href="https://colab.research.google.com/github/othniel77/datascience/blob/main/Pr%C3%A9vision_de_l'actio_Netflix_avec_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install keras_self_attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras_self_attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=e524783599610e4974998b413d3c1f71b8799c1fd3b3e01ce1c03ae691ddbbbb
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras_self_attention


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as smi
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tensorflow.keras.optimizers import Adam
from keras_self_attention import SeqSelfAttention
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
warnings.simplefilter('ignore')


In [10]:
from google.colab import files
data_to_load = files.upload()

Saving NFLX (2).csv to NFLX (2).csv


In [11]:
df = pd.read_csv('/content/NFLX (2).csv')
df = df[df['Date'] >= '2015-01-01']
df.head(5)


,Date,Open,High,Low,Close,Adj Close,Volume
3175,2015-01-02,49.151428,50.331429,48.731430,49.848572,49.848572,13475000
3176,2015-01-05,49.258572,49.258572,47.147144,47.311428,47.311428,18165000
3177,2015-01-06,47.347141,47.639999,45.661430,46.501431,46.501431,16037700
3178,2015-01-07,47.347141,47.421429,46.271427,46.742859,46.742859,9849700
3179,2015-01-08,47.119999,47.835712,46.478573,47.779999,47.779999,9601900


In [12]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1699 entries, 3175 to 4873
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1699 non-null   object 
 1   Open       1699 non-null   float64
 2   High       1699 non-null   float64
 3   Low        1699 non-null   float64
 4   Close      1699 non-null   float64
 5   Adj Close  1699 non-null   float64
 6   Volume     1699 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 106.2+ KB


## Construisons des paramètres pour la prédictions des séries temporelles et leur évaluation

In [14]:
data = df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)



In [15]:
def create_sequences(data, seq_length):
    sequences, labels = [], []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length, 0]
        label = data[i + seq_length, 0]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [16]:
seq_length = 4
n_samples = len(data_scaled)
sequences, labels = create_sequences(data_scaled, seq_length)
split = int(0.8 * len(sequences))
X_train, y_train = sequences[:split], labels[:split]
X_test, y_test = sequences[split:], labels[split:]


## LSTM (deep learning)

In [17]:
model_lstm = Sequential()
model_lstm.add(LSTM(100, activation='relu', input_shape=(seq_length, 1),return_sequences=True))
model_lstm.add(LSTM(50, activation='relu'))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse')


In [18]:
model_lstm.fit(X_train, y_train, epochs=20, batch_size=32,validation_data=(X_test, y_test))
predictions_lstm = model_lstm.predict(X_test)


Epoch 1/20
43/43 [==============================] - 4s 21ms/step - loss: 0.0618 - val_loss: 0.0045
Epoch 2/20
43/43 [==============================] - 0s 10ms/step - loss: 0.0028 - val_loss: 0.0274
Epoch 3/20
43/43 [==============================] - 0s 11ms/step - loss: 4.9523e-04 - val_loss: 0.0083
Epoch 4/20
43/43 [==============================] - 0s 10ms/step - loss: 3.2418e-04 - val_loss: 0.0025
Epoch 5/20
43/43 [==============================] - 0s 11ms/step - loss: 3.2297e-04 - val_loss: 0.0028
Epoch 6/20
43/43 [==============================] - 0s 11ms/step - loss: 3.0769e-04 - val_loss: 0.0024
Epoch 7/20
43/43 [==============================] - 1s 16ms/step - loss: 2.9517e-04 - val_loss: 0.0019
Epoch 8/20
43/43 [==============================] - 1s 16ms/step - loss: 2.8619e-04 - val_loss: 0.0023
Epoch 9/20
43/43 [==============================] - 1s 17ms/step - loss: 2.8662e-04 - val_loss: 0.0012
Epoch 10/20
43/43 [==============================] - 1s 18ms/step - loss: 2.9810e

In [19]:
predictions__LSTM = scaler.inverse_transform(predictions_lstm)
y_test_actual_LSTM = scaler.inverse_transform(y_test.reshape(-1, 1))


In [20]:
dates_test = df['Date'].iloc[split + seq_length:]
df_plot_lstm = pd.DataFrame({'Fechas': dates_test, 'Real': y_test_actual_LSTM.flatten(), 'Predict': predictions__LSTM.flatten()})
df_plot_lstm.head(1)

,Fechas,Real,Predict
4535,2020-05-29,419.730011,422.981232


## LSTM + Attention

In [21]:
model_lstm_attention = Sequential()
model_lstm_attention.add(LSTM(100, activation='relu', input_shape=(seq_length,1), return_sequences=True))
model_lstm_attention.add(SeqSelfAttention(attention_activation='relu'))
model_lstm_attention.add(LSTM(50, activation='relu', return_sequences=False))
model_lstm_attention.add(Dense(1))
model_lstm_attention.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [22]:
model_lstm_attention.fit(X_train, y_train, epochs=20, batch_size=32,validation_data=(X_test, y_test))
predictions_scaled_RNN = model_lstm_attention.predict(X_test)


Epoch 1/20
43/43 [==============================] - 6s 35ms/step - loss: 0.0629 - val_loss: 0.0247
Epoch 2/20
43/43 [==============================] - 1s 12ms/step - loss: 0.0030 - val_loss: 0.0047
Epoch 3/20
43/43 [==============================] - 1s 12ms/step - loss: 3.9818e-04 - val_loss: 0.0026
Epoch 4/20
43/43 [==============================] - 1s 12ms/step - loss: 3.2018e-04 - val_loss: 0.0018
Epoch 5/20
43/43 [==============================] - 1s 12ms/step - loss: 3.0163e-04 - val_loss: 0.0018
Epoch 6/20
43/43 [==============================] - 0s 11ms/step - loss: 3.0680e-04 - val_loss: 0.0015
Epoch 7/20
43/43 [==============================] - 1s 12ms/step - loss: 3.2110e-04 - val_loss: 0.0014
Epoch 8/20
43/43 [==============================] - 1s 12ms/step - loss: 3.1965e-04 - val_loss: 0.0013
Epoch 9/20
43/43 [==============================] - 1s 12ms/step - loss: 2.9134e-04 - val_loss: 0.0010
Epoch 10/20
43/43 [==============================] - 0s 11ms/step - loss: 3.0469e

In [23]:
predictions_lstm_attention = scaler.inverse_transform(predictions_scaled_RNN)
y_test_lstm_attention = scaler.inverse_transform(y_test.reshape(-1, 1))

In [24]:
df_plot_lstm_attention = pd.DataFrame({'Fechas': dates_test,
                                       'Real':y_test_lstm_attention.flatten(),
                                       'Predict': predictions_lstm_attention.flatten()})
df_plot_lstm_attention.head(1)


,Fechas,Real,Predict
4535,2020-05-29,419.730011,416.940247
